In [7]:

import pandas as pd
import seaborn as sns
import scipy
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

# Generation of standard input data for v1 model
## Experiment input data reconstruction
This Jupyter script estimates the prevalence of infectious Omicron-carriers in South Africa, since the start of detection of Omicron. This prevalence time-series data will be used in the ISIR-model, as the 'shape' of importation rates of Omicron-carriers into the NL.

From GISAID data, the first reported genomes of Omicron were from samples dating in the week of 1-7 November 2021*. The GISAID data also describes the proportion of Omicron genomes out of the submitted samples, which we shall use as proxy describing the proportion of Omicron cases out of all of SA's COVID-19 cases since early November.

We use the John Hopkins' CSSE Coronavirus Dashboard dataset for the daily reported COVID-19 cases within SA. The code of the data operations can be found at ``OFEM_PreSim1_CSSE_timeseries_SA_extract.ipynb``.

(TODO: describe the data operations)


\* Note that some literature state that the first case was on 9 November.
- we know when the first submission of Omicron was done (GISAID source)



## Assumptions
- Assumption: currently very linear (not based on viral load peaks)
- Assumption: uncounted cases is a constant, and not variable (e.g. due to increase/decrease in priority and testing rate)
- Assumption: does not account for changes in testing rate
- bias weeks by end of week (first known case is 9th Nov), because sampling numbers are reported on week of collection
- CHECK: whether dates are represented correctly (ie. first day of first case, vs first day of month). Very important for policy application and output, because those work with dates instead.

# Case numbers from CSSE: number of infectious people per day (proxy)
note: this code section does the following:
- apply a 'rolling inclusion' algorithm that lumps together cases from several days earlier and several days later, representing infectious people (TODO: cite papers here why)
- multiply with a 'underestimation correction' factor to represent undetected cases. Currently set to 10, from NICD modelling estimates prior. Caveats at ADD.C

note: some adjustments were made to the raw data:
- time axis biased earlier in time to have similar curve as the PDPC factsheet table, which is in turn based from the NICD's own statistics. Even so, the numbers are slightly different (see ADD.B)
- CSSE's correction of adding positive cases on a specific day was reversed due to very uncertain reasoning (see ADD.A)

## Addendum
- ADD.A: NICD announced a correction which led to a strong spike on one day. However, NICD's Twitter announcement on it is very unclear
- ADD.B: the CSSE data seems to have a 1-day lag in general shape compared to NICD's own documents. However, the numbers are not exactly the same, and doesn't seem to be corrected for undetected cases.
- ADD.C: the flat inflation factor is based on the NICD's modelling initiative at the start of the pandemic. In two updates, they mention an underestimation of 1:4 (detected vs total) to 1:10

In [8]:
# cases_dir = "../data_factsheet"
# cases_file = "SA_cases_nov2021.tsv"
# cases_df = pd.read_csv(f"{cases_dir}/{cases_file}", sep='\t', comment='#').set_index('day')
# # cases_df = cases_df.astype()  # for nullable format
# read from pre-sorted CSSE data
cases_df = pd.read_pickle(f"../data_input/data_CSSE_2021-10-15_2022-06-30.pickletable")

In [9]:
# cases_df.index = cases_df.index.shift(periods=-1, freq='D')  # (depreciated, time shift unnecessary?)
# Constant definition for reference
date_monthstart = '2021-11-01'  # start of november
date_1st_omi = '2021-11-09'  # first reported case of Omicron
date_reference = '2021-11-26'  # day of case study

In [10]:
# define weights from gamma function  (based on AdH serial interval distribution)
a = 2.14
rate = .59
x = range(0, 13, 1)  # ends on the 13th day (note pythonic count)
t_incubation = 3  # incubation day based on the UKHSA rapid review document (non-pythonic count)

func_serial = scipy.stats.gamma.pdf(x=x, a=a, scale=1 / rate,
                                    loc=-1)  # we set loc to be -1 for now (different from model input)
func_serial_norm = func_serial / sum(func_serial)  # normalise
# establish rolling window
rw_fore = t_incubation - 1  # ie. number of days after t_incubation
rw_aft = len(func_serial_norm) - t_incubation


In [11]:
## NOTE: MOVED AFT
# conduct rolling window
# expressing as nd-array numpy
# consider time range of validity
# infectious_rw_collect = {}
# for idx, val in zip(cases_df.index, cases_df.values):
#     date_fore = idx - pd.to_timedelta(arg=rw_fore, unit='D')  # get infectious 'power' before
#     date_aft = idx + pd.to_timedelta(arg=rw_aft, unit='D')  # get infectious 'power' after
#     infectious_rw_collect[idx] = pd.Series(val * func_serial_norm,
#                                            index=pd.date_range(
#                                start=date_fore,
#                                end=date_aft))
#
# infectious_rw_mat = pd.concat(infectious_rw_collect, axis=1)
#
#
# to_save = False
# # to save
# save_dir = '../data_input'
# name_irw_mat = 'infectious_func_rw_origin_matrix'
#
# if to_save:
#
#     infectious_rw_mat.to_pickle(f'{save_dir}/{name_irw_mat}.pickletable')
#     infectious_rw_mat.to_csv(f'{save_dir}/{name_irw_mat}.csv')

In [12]:
# sns.lineplot(infectious_rw_func)

In [13]:

# (depreciated)
# this block makes a rudimentary estimate of the current number of infectious persons
# 'count fore' assumes future daily cases are part of the infectious group
# count_pre_n = 3  # source Kim EA 2022 & internal
# count_post_n = 10  #
# INFL_FACTOR = 10  # (depreciated) inflation factor, assumed constant
#
# """ Rolling inclusion algorithm:
# Includes cases several days before and after the report date
# <A> also include last day (hence +1) """
# daily_sum = {}
# for idx in cases_df.index:
#     ilocidx = cases_df.index.get_loc(idx)
#     daily_sum[idx] = cases_df.iloc[ilocidx - count_post_n
#                                    :ilocidx + count_pre_n + 1].sum()  # <A>
#
# # NOTE to truncate fore and aft of the generate points, due to overflow TODO revise for clarity
#
# # create new DF for all cases in that time period
# infectious = pd.DataFrame.from_dict(daily_sum, orient='index')
#
# # apply inflation (depreciated)
# infectious_infl = infectious * INFL_FACTOR

# save to pickle (or leave for later)
# cases_df.to_pickle('data_input/data_cases_factsheet.pickletable')

In [14]:
# from interpolant of samples, we want to estimate the number of Omicron cases from cases numbers
# date_nominal = "2021-11-26"  # nominal policy date
# date_end = "2021-12-10"  # latest policy is 2 weeks after 26th, NOTE december


In [15]:
# gather dates of interest, starting on first observation (9th Nov 2021). End date is already truncated to 31st Jan 2022
# infectious_infl = infectious_infl.loc[date_1st_omi:, :]
# infectious_noinfl = infectious_rw_func.loc[date_1st_omi:]  # another for non-inflated numbers
#
# # convert from dates to number of days, for interpolation scheme to work. We start from 0 as 1st day (pythonic count) for consistency
# cases_days_elapsed = (infectious_infl.index - pd.to_datetime(date_1st_omi)).days
# date_day_ref = dict(zip(days_elapsed, cases_daily_roll.index))


# Samples from GISAID: proportion of Omicron in population (by proxy)
note (expand later)
- week bias forward
- assumption of continuousness (and not noisy)

In [16]:
# read data file
samples_dir = '../../data_gisaid'
samples_file = 'omicron_samples_SA_long.pickletable'
samples_df = pd.read_pickle(f'{samples_dir}/{samples_file}')
# Bias weeks forward because of weekly sampling reporting (samples for the week are reported on the subsequent Sunday)
samples_df.index = pd.to_datetime(samples_df.index) #- pd.to_timedelta(arg=1, unit='W')

# Add data points for interpolation work
samples_df.loc[pd.to_datetime(samples_df.index[0] - pd.to_timedelta(arg=1, unit='W'))] = [0, 0, 0]
# samples_df.loc[pd.to_datetime(date_1st_omi) - dt.timedelta(days=1)] = [0, 0, 0]
# ^ set day before 1st Omicron to be 0 for interpolation
samples_df = samples_df.sort_index()  # sort to be in correct order again

# Convert dates to days again
samples_df['delta'] = pd.to_timedelta(
    pd.to_datetime(samples_df.index) - pd.to_datetime(date_reference)
    , unit='D').days  # converts to days, where omicron 1st observation = 0

# save samples to model input folder
# samples_df.to_pickle('data_input/omicron_samples_SA_mod1.pickletable')

# Estimating Omicron cases in SA
This section estimates the number of Omicron cases in SA starting from the first day of observation (9th Nov 2021), via the following steps:
1. As Omicron was competing with Delta, the share of Omicron cases increases per day in SA. This is represented by an interpolation function based on the weekly sampling data, which estimates the proportion of Omicron cases amongst the daily COVID cases.
2. Multiply this proportion information with the SA daily case statistics, to get the number of infectious Omicron carriers per day.

NOTE!: the ordering of inflation before interpolation may be incorrect. This might be revised later.

In [0]:
# create interpolation function, using delta time [days since 1st omicron case].
# 'linear' setting because of weird harmonics from quadratic fitting.
omicron_ratio_fx = interp1d(x=samples_df['delta'], y=samples_df['ratio'], kind='linear', bounds_error=False)

In [21]:
# MOD
infectious_rw_func = cases_df.to_frame('infectious_cases')
infectious_rw_func['day_rel'] = (infectious_rw_func.index - pd.to_datetime(date_reference)).days
infectious_rw_func['omi_pct'] = omicron_ratio_fx(infectious_rw_func['day_rel'])

infectious_rw_func['infect_pax'] = infectious_rw_func['infectious_cases'] * infectious_rw_func['omi_pct']
# infectious_rw_func2['infect_pax2'] =

# apply rolling window?



In [22]:
# conduct rolling window, after the calculation of Omicron cases
# expressing as nd-array numpy
# consider time range of validity
# We assume that new cases are reported on the 3rd day of the AdH&B curve (ie. start of symptomatic period)
infectious_rw_collect = {}
work_series = infectious_rw_func['infect_pax']
for idx, val in zip(work_series.index, work_series.values):
    date_fore = idx - pd.to_timedelta(arg=rw_fore, unit='D')  # get infectious 'power' before
    date_aft = idx + pd.to_timedelta(arg=rw_aft, unit='D')  # get infectious 'power' after
    # calculate the 'infectious power' of current individuals within their infectious window
    infectious_rw_collect[idx] = pd.Series(val * func_serial_norm,
                                           index=pd.date_range(
                               start=date_fore,
                               end=date_aft))

infectious_rw_mat = pd.concat(infectious_rw_collect, axis=1)


to_save = True
# to save
save_dir = '../data_input'
name_irw_mat = 'infectious_func_rw_origin_matrix_v2'  # v2 is a rearrangement, such that the rolling window is applied after the number of Omicron cases are determined

if to_save:

    infectious_rw_mat.to_pickle(f'{save_dir}/{name_irw_mat}.pickletable')
    infectious_rw_mat.to_csv(f'{save_dir}/{name_irw_mat}.csv')

In [23]:
# Get the 'infectious presence' # TODO what is infectious presence
infectious_rw_func['infect_presence'] = infectious_rw_mat.sum(axis=1)

In [26]:
# DEPRECIATED, CAN BE DELETED
# cases_days_elapsed = range(samples_df['delta'].min(), samples_df['delta'].max() + 1) # +1 due to pythonic count
# apply interpolation to get proportion of Omicron infectious per day
# infectious_rw_func = infectious_rw_mat.sum(axis=1).to_frame('infectious_cases')
# infectious_rw_func['day_rel'] = (infectious_rw_func.index - pd.to_datetime(date_reference)).days
# infectious_rw_func['omi_pct'] = omicron_ratio_fx(infectious_rw_func['day_rel'])
#
# infectious_rw_func['infect_pax'] = infectious_rw_func['infectious_cases'] * infectious_rw_func['omi_pct']

name_irw_func = 'infectious_func_rw_rev3'
# infectious_rw_func = infectious_rw_func.dropna(subset=['omi_pct','infect_pax'])  # drop NA things, depreciated
infectious_rw_func = infectious_rw_func.loc[(infectious_rw_func['infect_presence'] > 0).idxmax():,:] # trim out early rows where no Omicron infectious power is available
infectious_rw_func.to_csv(f'{save_dir}/{name_irw_func}.csv')  # save raw data
infectious_rw_func = infectious_rw_func.reset_index(names='date')
infectious_rw_func.to_pickle(f'{save_dir}/{name_irw_func}.pickletable')

In [10]:
# simple test plot
# x = np.arange(interp_f1.x.min(), interp_f1.x.max())
# plt.plot(x, interp_f1(x), label='linear')
# plt.plot(x,interp_f2(x), label='quadratic')
# plt.legend()

# cold storage

In [11]:
infectious_omi = infectious_infl.squeeze() * infectious_omi_pct  # squeeze to coerce into Series, otherwise error
infectious_omi_noinfl = infectious_noinfl.squeeze() * infectious_omi_pct
# find prevalence in SA
pop_sa = int(60.142978e6)  # from factsheet
flightpax_direct = 4839  # from Schiphol document
flightpax_indirect = 980  # ditto
flightpax_transfer = 7303  # ditto, but only transferring people through Schiphol

flightpax_daily = (flightpax_direct + flightpax_indirect) / 30  # from schiphol, direct flights
prevalence_omi = infectious_omi / pop_sa
flightpax_omi = prevalence_omi * flightpax_daily

# also create for uninflated versions
prevalence_omi_noinfl = infectious_omi_noinfl / pop_sa
flightpax_omi_noinfl = prevalence_omi_noinfl * flightpax_daily

In [12]:
# flightpax_omi.loc["2021-11-09":"2021-11-25"].sum() * 2
flightpax_omi_noinfl.loc["2021-11-09":"2021-11-25"].sum()


0.15385308751350346

In [13]:
flightpax_omi_noinfl_elapsed = flightpax_omi_noinfl.to_frame(name='import_pax')
flightpax_omi_noinfl_elapsed['prevalence'] = prevalence_omi_noinfl
flightpax_omi_noinfl_elapsed['day'] = cases_days_elapsed.values
flightpax_omi_noinfl_elapsed = flightpax_omi_noinfl_elapsed.reset_index(names='date').set_index('day')
flightpax_omi_noinfl_elapsed.to_pickle('data_input/omi_prevalence_flights_not_inflated.pickletable')

In [14]:
# create dumb generator
# prob_roll = np.random.rand(prevalence_omi.size,int(round(flightpax_daily)))

In [15]:
# np.less(prob_roll,prevalence_omi.to_numpy()[:,np.newaxis]).sum(axis=1)
# for ridx, prob in enumerate(prevalence_omi):



#  Preliminary build for class-style construction
Cold storage for now

In [16]:
# from typing import Callable, Dict, Tuple
#
# # this class should be refactored later for experimental design, for efficiency
# class OI_importer_build:
#
#     def __init__(self, path_samples):
#         # putting down some known variables
#         # self.date_ref : str  # default, is it necessary?
#         self.path_samples : str = path_samples # default
#         self.path_cases : str  # default
#         self.daily_pax_flights: int|Dict # number of inbound passengers, either constant or dict/function (future)
#         self.daily_imports: float
#         self.samples_tbias: int  # time bias for sampling curve fore/aft of data
#         self.cases_tbias: int # time bias for cases
#         self.cases_tinclude: int # time inclusion for fore/aft cases. May extend to tuple for both fore and aft inclusion (which might need a custom function)
#
#         self.omi_ratio_input = None # inputs for the interp1d
#         self.omi_ratio_curve = None # function itself, output of samples submodule
#         self.omi_ratio_daily = None # dict lookup
#         self.cases_lookup = None # lookup (prolly dict) of daily cases emerging
#         self.t_restrict: int|None # day (delta) on which flights are restricted
#
#         pass
#
#     def generate_omicron_ratios(self):
#         # read from pickle and generate interp1d function and dict
#         self.omi_ratio_input = pd.read_pickle(self.path_samples)
#         pass